# Lab 1 - Creating a Slice with a P4-DPDK Pipeline
<div style="text-align: justify;">
This lab will walk you through creating an experiment that contains a P4-DPDK programmable pipeline. Specifically, the experiment deploys a slice in a single site with one node: server1. The topology is shown in the figure below. The lab provides an introduction to DPDK, a software-based packet processing acceleration tool. It demonstrates how to build a topology using namespaces and provides an explanation of scripts needed to build a P4-DPDK pipeline. </div>

<figure style="text-align: center;">
    <img src="./labs_files/lab1/figs/0_00_fabric_topology.png" width="550px"><be>
</figure>

# Background

<div style="text-align: justify;">
The process of receiving and handling network packets involves several steps, each of which contributes to the overall system performance. Accelerating packet processing while minimizing overhead is crucial for achieving optimal network throughput. Before packet processing accelerators, a standard Network Interface Card (NIC) receives a data packet (e.g., from the network cable), and triggers an interrupt. This interrupt is a signal sent to the operating system (OS) to inform it that data has arrived. The OS then processes this interrupt and identifies the memory location where the packet is stored. This location is typically in the system’s Random Access Memory (RAM). The network stack is a software component within the OS responsible for handling network-related tasks. It includes protocols, drivers, and other networking functionalities. Once the OS knows the packet’s location in memory, it transfers the packet to the network stack. This transfer involves copying the packet data from the NIC’s buffer to the network stack’s buffer. To deliver the packet to the correct user-level application (e.g., a web browser or email client), the network stack relies on system calls [<a href="#References">1</a>]. </div>  <br>

<div style="text-align: justify;">
While the above steps of standard packet processing shown in Figure 1 (a) are necessary for proper packet handling, they come at a cost. The additional processing involved in interrupt handling, memory copying, and system calls creates overhead. To mitigate this, modern systems use techniques like interrupt moderation, zero-copy networking, and kernel bypass mechanisms to minimize the impact of overhead on network performance. </div>   <br>

<div style="text-align: justify;">
The Data Plane Development Kit (DPDK) is a software packet processing acceleration tool that consists of a collection of libraries and drivers that support packet processing within the user space while bypassing the kernel as shown in Figure 1 (b). With DPDK, the ports of the NIC do not rely on the in-kernel drivers. Instead, a NIC that supports DPDK is managed by DPDK drivers that operate as a Poll Mode Driver (PMD). It receives, classifies, and delivers packets as it consistently polls for incoming packets [<a href="#References">2</a>]. This approach minimizes interrupt services overhead and improves performance. DPDK functionalities can also run on multiple cores with specific tasks running on each core using core affinity which prevents task switching among different cores and therefore enhances performance [<a href="#References">1</a>]. </div>   <br>

<figure style="text-align: center;">
  <img src="./labs_files/lab1/figs/0_intro_1_dpdk.png" width="550" style="display: block; margin: 0 auto;">
  <figcaption>Figure 1. Software packet processing. (a) standard packet processing (interrupt-based), (b) kernel-bypass packet processing (polling mode) [<a href="#References">2</a>].</figcaption>
</figure>

<div style="text-align: justify;">
In network processing, both the CPU and the NIC frequently require access to data stored in memory such as cache and Dynamic Random Access Memory (DRAM). To optimize memory access, DPDK supports the use Hugepage and memory pools. To serve memory management purposes, DPDK swiftly moves data into the cache to prevent CPU overheads [<a href="#References">3</a>]. </div>

## DPDK pipeline model

<div style="text-align: justify;">
The DPDK Packet Framework allows the implementation of accelerated packet processing with great flexibility. This framework required the use of a DPDK library (librte_pipeline) which provides a methodology for building a programmable pipeline tailored to serve a specific application [<a href="#References">1</a>]. These pipelines act as modular building blocks that can be interconnected through packet queues to create entire network applications. See Figure 2. A DPDK pipeline has three main components: input ports, tables, and output ports. Each pipeline can be instantiated multiple times, with each instance mapped to a different CPU thread [<a href="#References">4</a>]. </div>   <br> 

<figure style="text-align: center;">
  <img src="./labs_files/lab1/figs/0_intro_2_pipeline.png" width="550" style="display: block; margin: 0 auto;">
  <figcaption>Figure 2. DPDK packet framework pipeline block [<a href="#References">5</a>].</figcaption>
</figure>

## P4 programming

<div style="text-align: justify;">
Although DPDK is based on C programming, writing in P4 is generally considered more straightforward. P4 is a programming language to control the packet processing within the data plane of programmable forwarding elements, such as hardware or software switches, network interface cards, routers, and various network devices. Although initially designed for programmable switches, P4's application has expanded to be compatible with a diversity of devices called P4 targets, including SmartNICs. P4 is specifically designed to program the data plane of the target. The P4 code is written by the user in a specific architecture to ensure compatibility with the target. Afterward, the P4 code is ready to be compiled so that it can be executed by the target [<a href="#References">6</a>]. </div>   <br> 

## P4-DPDK compiler

<div style="text-align: justify;">
The Software Switch (SWX) pipeline integrates DPDK performance with the flexibility of the P4 language. It serves as a tool for developing software switches or data plane applications. Moreover, it can be combined with the open-source P4 compiler p4c-dpdk. This enables the translation of P4 programs to the DPDK SWX API, allowing them to run efficiently on multi-core CPUs. The primary output of the p4c-dpdk compiler given a P4 code is the specifications file (.spec). This file is needed to configure the DPDK pipeline. Subsequently, a C code is generated from the .spec file. This code includes C functions corresponding to each action and control block. A C compiler then generates a shared object (.so) from the C code. Finally, the shared object is needed to execute the application [<a href="#References">7</a>]. </div>   <br> 

<figure style="text-align: center;">
  <img src="./labs_files/lab1/figs/0_intro_3_p4dpdk.png" width="550" style="display: block; margin: 0 auto;">
  <figcaption>Figure 3. The p4c-dpdk workflow.</figcaption>
</figure>

## Memory in DPDK

<div style="text-align: justify;">
Memory management is very important to maintain performance measures. DPDK supports various memory management features such as Hugepages, Non-uniform Memory Access (NUMA) nodes pinning, and memory pools [<a href="#References">8</a>]. </div>   <br> 

<div style="text-align: justify;">
A hugepage is a memory management technique used in modern computer systems to improve performance by using larger memory blocks (pages) than the default page size. 
When the DPDK application initializes and requests a certain number of hugepages, the operating system will reserve a large block of memory and allocate it to the application. Hugepage reservation is required in packet processing applications due to the large memory pool allocation used for packet buffers. If regular memory is used instead, applications using DPDK would experience significant performance degradation due to the high rate of accessed memory location misses. </div>   <br> 

<div style="text-align: justify;">
NUMA node pinning is a technique used in computer systems with NUMA architecture to optimize performance by controlling how processes and memory are allocated. In a NUMA system, CPU cores are grouped into nodes, with each node having its own dedicated memory that is physically close to the running CPU cores. A general representation of the system is shown in Figure 4. Memory management features offered by DPDK make it less probable to write a poorly performing user application by including APIs where NUMA nodes have to be specified to run the applications. Therefore, pinned NUMA nodes to CPU cores are configured for every operation being held by the system. </div>   <br> 

<figure style="text-align: center;">
  <img src="./labs_files/lab1/figs/0_intro_4_numa.png" width="500" style="display: block; margin: 0 auto;">
  <figcaption>Figure 4. NUMA node pinning [<a href="#References">8</a>].</figcaption>
</figure>

# Step 1:  Configuring the environment

Before running this notebook, you will need to configure your environment using the [Configure Environment](../../../configure_and_validate.ipynb) notebook. Please stop here, open and run that notebook, then return to this notebook.

If you are using the FABRIC JupyterHub many of the environment variables will be automatically configured for you.  You will still need to set your bastion username, upload your bastion private key, and set the path to where you put your bastion private key. Your bastion username and private key should already be in your possession.  

If you are using the FABRIC API outside of the JupyterHub you will need to configure all of the environment variables. Defaults below will be correct in many situations but you will need to confirm your configuration.  If you have questions about this configuration, please contact the FABRIC admins using the [FABRIC User Forum](https://learn.fabric-testbed.net/forums/) 

More information about accessing your experiments through the FABRIC bastion hosts can be found [here](https://learn.fabric-testbed.net/knowledge-base/logging-into-fabric-vms/).
 

# Step 2: Importing the FABlib library

In [1]:
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager
fablib = fablib_manager()

# Step 3: Creating the experiment slice

The following creates a node with basic compute and networking capabilities. You build a slice by creating a new slice and adding resources to the slice. After you build the slice, you must submit a request for the slice to be instantiated.   

### Step 3.1: Creating a slice
The code below creates a new slice with the name "P4DPDK_lab1_vnic"

In [2]:
slice = fablib.new_slice(name="P4DPDK_lab1_vnic")

### Step 3.2: Defining the sites
The code below requests a random site from FABRIC based on the condition that the following resources are available:

<ul>
    <li> 4 CPU cores</li>
    <li> 8GB RAM </li>
    <li> 20GB disc size
</ul>

In [3]:
site1= fablib.get_random_sites(count=1, filter_function=lambda x: x['cores_available'] > 4 and x['ram_available'] > 8 and x['disk_available'] > 20)[0]

print (f'The selected site is {site1}')

The selected site is MAX


### Step 3.3: Creating the nodes
The code below creates one node (server1) which uses the following:
<ul>
    <li> 4 CPU cores</li>
    <li> 8GB RAM </li>
    <li> 20GB disc size </li>
    <li> Image: Ubuntu 20.04
</ul>

server1 will be created in site1

In [4]:
server1 = slice.add_node(name="server1", 
                      site=site1, 
                      cores=4, 
                      ram=8, 
                      disk=20, 
                      image='default_ubuntu_20')

### Step 3.4: Submitting the slice
The code below submits the slice. 
By default, the submit function will block until the node is ready and will display the progress of your slice being built.

In [5]:
slice.submit();


Retry: 9, Time: 205 sec


ID,7a33bf92-fc75-4525-8ec5-2d52d40c3c3d
Name,P4DPDK_lab1_vnic_test
Lease Expiration (UTC),2024-09-21 17:22:38 +0000
Lease Start (UTC),2024-09-20 17:22:38 +0000
Project ID,8eaa3ec2-65e7-49a3-8c09-e1761141a6ad
State,StableOK


ID,Name,Cores,RAM,Disk,Image,Image Type,Host,Site,Username,Management IP,State,Error,SSH Command,Public SSH Key File,Private SSH Key File
dd8d00ce-0eda-4476-b82d-4eec26532504,server1,4,8,100,default_ubuntu_20,qcow2,max-w1.fabric-testbed.net,MAX,ubuntu,2001:468:c00:ffc4:f816:3eff:fe4f:3192,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2001:468:c00:ffc4:f816:3eff:fe4f:3192,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key


# Step 4: Installing the required packages
In this step, we will install the required packages to run the lab. Specifically, we will install the DPDK library, the P4 compiler (p4c), and all needed dependencies.

### Step 4.1 Getting the server node
The command below gets the fablib node that server1 is associated with.

In [6]:
server1 = slice.get_node(name="server1")

### Step 4.2 NAT64 setup
The code below checks if an IPv6 address is available to set up NAT64. We will upload the script [scripts/nat64.sh](./scripts/nat64.sh) to the all servers and execute it

In [7]:
from ipaddress import ip_address, IPv6Address

if type(ip_address(server1.get_management_ip())) is IPv6Address:
    server1.upload_file('scripts/nat64.sh', 'nat64.sh')
    stdout, stderr = server1.execute(f'chmod +x nat64.sh && ./nat64.sh', quiet=True)

### Step 4.3 Installing dependencies
The code below installs packages that are prerequisites to the upcoming installations and needed to run the lab experiments

In [8]:
stdout, stderr = server1.execute(f'sudo apt-get update', quiet = True)
stdout, stderr = server1.execute(f'sudo apt-get install -y build-essential python3-pip python3-pyelftools libnuma-dev pkg-config net-tools', quiet = True)
stdout, stderr = server1.execute(f'sudo pip3 install meson ninja', quiet = True)

### Step 4.4 Installing DPDK
The code below downloads, builds, and installs DPDK on all servers. In this lab, we are building a modified version of DPDK in which we enabled logs in the terminal to have a better understanding of the behavior of the built pipeline. Note that printing out logs on the terminal degrades performance. Therefore, with applications where high performance is needed, logging should be disabled.  

In [9]:
stdout, stderr = server1.execute(f'git clone http://dpdk.org/git/dpdk', quiet = True)
stdout, stderr = server1.execute(f'cd dpdk/lib/pipeline/ &&  sudo rm rte_swx_pipeline.c && sudo rm rte_swx_pipeline_internal.h', quiet = True)
server1.upload_file('scripts/rte_swx_pipeline.c','/home/ubuntu/dpdk/lib/pipeline/rte_swx_pipeline.c')
server1.upload_file('scripts/rte_swx_pipeline_internal.h','/home/ubuntu/dpdk/lib/pipeline/rte_swx_pipeline_internal.h')
stdout, stderr = server1.execute(f'cd dpdk &&  sudo meson build && cd build && sudo ninja && sudo ninja install && sudo ldconfig', quiet = True)
stdout, stderr = server1.execute(f'cd dpdk/examples/pipeline && sudo make', quiet=True)

### Step 4.5 Install p4c
The code below downloads and installs the p4c compiler needed to compile the p4 code into a DPDK pipeline

In [10]:
stdout, stderr = server1.execute('source /etc/lsb-release && echo "deb http://download.opensuse.org/repositories/home:/p4lang/xUbuntu_${DISTRIB_RELEASE}/ /" | sudo tee /etc/apt/sources.list.d/home:p4lang.list && curl -fsSL https://download.opensuse.org/repositories/home:p4lang/xUbuntu_${DISTRIB_RELEASE}/Release.key | gpg --dearmor | sudo tee /etc/apt/trusted.gpg.d/home_p4lang.gpg > /dev/null && sudo apt-get update && sudo apt install -y p4lang-p4c', quiet = True)
stdout, stderr = server1.execute("cd /usr/share/p4c/p4include/dpdk/ && sudo sed -i '769s/\\(.\\{6\\}\\)/\\1out/' pna.p4", quiet=True)

# Step 5: Implementing the P4-DPDK pipeline
This section shows the steps required to implement a P4-DPDK pipeline. It discusses the components needed for the process of compiling, building, and running the pipeline.

## Step 5.1: Compiling the P4-DPDK pipeline
The P4 compiler that will be used is p4c-dpdk, which transforms the P4 code and executes it into a DPDK pipeline.

### Step 5.1.1: Inspect P4 code
Click on [lab1.p4](./labs_files/lab1/lab1.p4) to open the file in the editor.

<img src="./labs_files/lab1/figs/5_01_p4.png" width="300px"><br>

In this lab, we will not modify the P4 code. Instead, we will just compile it using the p4c-dpdk compiler. Note that in this P4 code the Portable NIC Architecture (PNA) is used as shown in the grey box.

### Step 5.1.2: Compile the P4 code
To upload and compile the P4 program, issue the following command.

In [11]:
server1.upload_file('labs_files/lab1/lab1.p4','lab1.p4')
stdout, stderr = server1.execute(f'sudo p4c-dpdk --arch=pna lab1.p4 -o lab1.spec')
stdout, stderr = server1.execute(f'ls')

 sudo: unable to resolve host server1: Name or service not known
 dpdk
lab1.p4
lab1.spec
nat64.sh


The command above invokes the ```p4c-dpdk``` compiler to compile the '''lab1.p4''' program which is compatible with the PNA architecture as specified after the ```--arch``` flag. After executing the command, if there are no messages displayed, then the P4 program was compiled successfully. We will see in the printed list the ```lab1.spec``` file, which is a specification file generated by the p4c-dpdk compiler in the current directory as specified after the ```-o``` flag. 
Now that we have compiled the P4 program and generated the spec file, we can create the P4-DPDK pipeline.

## Step 5.2: Preparing the P4-DPDK CLI script
Each P4-DPDK pipeline is built through the CLI script. In this subsection, we will write the CLI script in which the pipeline is created and built.

Click on [lab1.cli](./labs_files/lab1/lab1.cli) to open the CLI file in the editor.

<img src="./labs_files/lab1/figs/5_02_01.png" width="650px"><br>
We can see that the lab1.cli file is empty and we have to fill it.

<hr>

We will start by generating the pipeline code and building the shared object. Write the following in the lab1.cli file.

    pipeline codegen /home/ubuntu/lab1.spec /tmp/lab1.c
    pipeline libbuild /tmp/lab1.c /tmp/lab1.so

<img src="./labs_files/lab1/figs/5_02_02.png" width="650"><br>

In the figure above the ```codegen``` function (line 2) is used to generate the C code of the compiled user application in the spec file. This function takes two arguments, the path to the specification file compiled ```/home/ubuntu/lab1.spec``` and the generated code is placed in a temporary directory ```/tmp/lab1.c```.

The ```libbuild``` function (line 3) is used to generate a shared object to execute the application. This function takes two arguments, the path to the C code ```/tmp/lab1.c``` and the generated shared object is placed in a temporary ```/tmp/lab1.so```.

<hr>

Now we will list the DPDK devices with customized parameters that match our setup. Write the following in the lab1.cli file.

    mempool MEMPOOL0 meta 0 pkt 9128 pool 32K cache 256 numa 0
    ethdev net_tap0 rxq 1 128 MEMPOOL0 txq 1 512 promiscuous on 
    ethdev net_tap1 rxq 1 128 MEMPOOL0 txq 1 512 promiscuous on


<img src="./labs_files/lab1/figs/5_02_03.png" width="650"><br>

The block of code in the figure above (lines 6-8) is used to create DPDK objects like a memory pool mempool and ethernet devices ethdev with parameters to setup each DPDK device.  A memory pool MEMPOOL0 is defined as follows: 

•	```mempool```: create a memory pool object associated with a given name. <br>
•	```meta```: specifies the private size of the memory buffer in bytes, which is the memory allocated for an application’s private data.<br>
•	```pkt```: specifies the private size of the memory buffer in bytes, which is the memory allocated for an application to store data associated with a packet.<br>
•	```pool```: the size of the defined memory pool specified in bytes.<br>
•	```cache```: the cache size in bytes which should be a power of 2. <br>
•	```numa```: pinned NUMA node ID.

Two ethernet devices which are the interfaces linked to the pipeline, are also defined as net_tap0 and net_tap1 as follows:

•	```ethdev```: ethernet device name (the attached devices net_tap0 and net_tap1 are virtual ethernet devices with their instances created when the pipeline is invoked). <br>
•	```rxq```: receiving queue parameters (number of receiving queues, queue size (bytes), memory pool name).<br>
•	```txq```: transmitting queue parameters (number of transmitting queues, queue size (bytes)). <br>
•	```promiscuous```: A mode that allows a network device to read each network packet that arrives (on / off).<br>

<div style="background-color: #e0f7fa; border: 1px solid #b2ebf2; padding: 10px; border-radius: 5px;">
It is important that the interface IDs net_tap0 and net_tap1 remain consistent as they will also be used in the I/O specification file and as they will be created while running the pipeline. This is the notation considered by DPDK while the operating system assigns different tags to the interfaces. 
</div>

<hr>

Now we will list the P4-DPDK pipelines. Write the following in the lab1.cli file.

    pipeline PIPELINE0 build lib /tmp/lab1.so io /home/ubuntu/ethdev.io numa 0

<img src="./labs_files/lab1/figs/5_02_04.png" width="650"><br>

In the figure above the ```build``` function (line 11) is used to create a pipeline object ```PIPELINE0```. This function takes the path of the shared object library ```lib  /tmp/lab1.so```, the path of the I/O file (which will be discussed in detail in the next subsection) ```io  /home/ubuntu/ethdev.io``` and the numa node ID ```numa  0```.

<hr>

Now we will map the created pipeline to a CPU thread. Write the following in the lab1.cli file.

    pipeline PIPELINE0 enable thread 1

<img src="./labs_files/lab1/figs/5_02_05.png" width="650"><br>

In the figure above the ```enable thread``` function (line 14) is used to map the pipeline ```PIPELINE0``` to the CPU thread ID 1.

<hr>

Save the changes by pressing ```Ctrl+s```.

## Step 5.3: Preparing the I/O script
The stream of packets within a P4-DPDK must be configured. In this subsection, we will write the I/O script which is the configuration file for the pipeline input and output stream.

Click on [ethdev.io](./labs_files/lab1/ethdev.io) to open the I/O file in the editor.

<img src="./labs_files/lab1/figs/5_03_01.png" width="650px"><br>

We can see that the ethdev.io file is empty and we have to fill it.

<hr>

We will start by defining the pipeline input ports. Write the following in the ethdev.io file.

    port in 0 ethdev net_tap0 rxq 0 bsz 1
    port in 1 ethdev net_tap1 rxq 0 bsz 1

<img src="./labs_files/lab1/figs/5_03_02.png" width="650px"><br>

Input ports to the pipeline are defined as follows:

•	```port in```: the pipeline input port ID<br>
•	```ethdev```: the ethernet device associated with the defined port<br>
•	```rxq```: the receiving queue ID<br>
•	```bsz```: burst size (packets)<br>

In the figure above the ```port in``` function (lines 2-3) is used to define two input ports to the pipeline, each from an interface. In this case, 0 and 1 are assigned as the port IDs of the first and second port respectively. ```net_tap0``` and ```net_tap1``` are both virtual ethernet devices that are associated with ports 0 and 1 respectively. 

Every packet received at an input port is then forwarded to a receiving queue in the pipeline as determined by the ```rxq``` parameter which holds a value representing the receiving queue ID. Both ports will forward packets to a single queue with ID 0. The ```bsz``` parameter represents the burst size. DPDK attempts to aggregate the cost of processing each packet individually by processing packets in bursts but in this experiment, the burst size is set to 1.

<hr>

Now we will define the pipeline output ports. Write the following in the ethdev.io file.

    port out 0 ethdev net_tap0 txq 0 bsz 1
    port out 1 ethdev net_tap1 txq 0 bsz 1

<img src="./labs_files/lab1/figs/5_03_03.png" width="650px"><br>

Output ports to the pipeline are defined as follows:

•	```port out```: the pipeline output port ID<br>
•	```ethdev```: the ethernet device associated with the defined port<br>
•	```txq```: the transmitting queue ID<br>
•	```bsz```: burst size (packets)<br>

In the figure above the ```port out``` function (lines 6-7) is used to define two output ports from the pipeline, each to an interface. Similar to the port in function, when this function is called, it is followed by the port ID and ethernet device interface ID. Every packet delivered at an output port is then forwarded to a transmitting queue in the pipeline as determined by the ```txq``` parameter which holds a value representing the transmitting queue ID. Both ports will forward packets to a single queue with ID 0. The burst size ```bsz``` is set to 1.

<hr>

Save the changes by pressing ```Ctrl+s```.

## Step 5.4: Running the P4-DPDK pipeline
Now that all the required scripts are prepared, we can run the pipeline.

### Step 5.4.1: Uploading files
The following code uploads the CLI and I/O scripts to server1.

In [13]:
server1.upload_file('labs_files/lab1/lab1.cli','lab1.cli')
server1.upload_file('labs_files/lab1/ethdev.io','ethdev.io')

<SFTPAttributes: [ size=217 uid=1000 gid=1000 mode=0o100664 atime=1726853902 mtime=1726853902 ]>

### Step 5.4.2: Reserving hugepages
Configure the number of hugepages in the system by typing the following command. 

In [14]:
stdout, stderr = server1.execute(f' sudo sh -c  "echo 1024 > /sys/kernel/mm/hugepages/hugepages-2048kB/nr_hugepages"')

 sudo: unable to resolve host server1: Name or service not known
 

Hugepage reservation is done by setting the number of hugepages required to the ```nr_hugepages``` file in the kernel corresponding to a specific page size (in Kilobytes).

The ```echo``` command is used to print a value which in this case is ```1024``` representing the number of hugepages. The ```>``` symbol is a redirection operator that redirects the output of the previous command (echo 1024) to the file specified in the following path: ```/sys/kernel/mm/hugepages/hugepages-2048kB/nr_hugepages```


### Step 5.4.3: Opening a terminal

Launch a new terminal by opening a new tab and then select "terminal".

<img src="./labs_files/lab1/figs/5_04_03_terminal.gif" width="800px"><br>

Copy the output of the command below and paste it into the terminal to enter server1.

In [15]:
server1.get_ssh_command()

'ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2001:468:c00:ffc4:f816:3eff:fe4f:3192'

### Step 5.4.4: Running the pipeline

Run the following commands in the terminal:
    
    cd dpdk
    sudo examples/pipeline/build/pipeline -c 0x3 --vdev=net_tap0,mac="00:00:00:00:00:01" --vdev=net_tap1,mac="00:00:00:00:00:02" -- -s /home/ubuntu/lab1.cli
    
<img src="./labs_files/lab1/figs/5_04_04.png" width="1000"><br>

In the figure above, the command is used to run the DPDK pipeline application considering the following arguments:

•	```examples/pipeline/build/pipeline```: the path to the executable DPDK pipeline application.<br>
•	```-c```: this parameter is used to specify the hexadecimal bitmask of the cores to run on. In this case, (0x3) indicated that 2 cores are reserved for the pipeline and one extra core is needed for other processes.<br>
•	```--vdev```: this parameter is used to create a virtual device also called a software NIC. Two virtual devices are created, ```net_tap0``` and ```net_tap1```.<br>
•	```mac```: fixed MAC addresses are assigned as ```00:00:00:00:00:01``` for net_tap0 and ```00:00:00:00:00:02``` for net_tap1. (If this parameter is not specified, random MAC addresses will be assigned to the virtual devices).<br>
•	```-s```: this parameter is used to specify the path to the CLI script file to be run at application startup ```/home/ubuntu/lab1.cli```.<br>


## Step 5.5: Inspecting interfaces

### Step 5.5.1: Opening a new terminal

Launch a new terminal by opening a new tab and then select "terminal".

<img src="./labs_files/lab1/figs/5_05_01.gif" width="900px"><br>

Copy the output of the command below and paste it into the terminal to enter server1.

In [16]:
server1.get_ssh_command()

'ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2001:468:c00:ffc4:f816:3eff:fe4f:3192'

### Step 5.5.2: Inspecting terminals

Run the following commands in the terminal:
    
    ifconfig
    
<img src="./labs_files/lab1/figs/5_05_02.png" width="600px"><br>

We can see that two new interfaces are displayed (highlighted in the grey boxes); ```dtap0``` and ```dtap1```. These interfaces are the interfaces of the virtual devices created earlier net_tap0 and net_tap1. The ```ifconfig``` command displays the names of the interfaces as understood by the operating system. 


### Step 5.5.3: Inspecting terminals in the pipeline

Enter the pipeline CLI by typing the following commands:
    
    telnet 0.0.0.0 8086
    ethdev show
    
<img src="./labs_files/lab1/figs/5_05_03.png" width="400px"><br>

The ```telnet``` command is followed by the broadcast IP address (0.0.0.0) and the port number (8086) to connect to the pipeline.

We can see that two interfaces are displayed (highlighted in the grey boxes); ```net_tap0``` and ```net_tap1```. These interfaces are the interfaces of the virtual devices created in the command that runs the pipeline. 

As we inspect the output for each ethernet device in the figure above, we can see in the first line, that ```ether``` is the MAC addresses assigned to the interfaces while running the pipeline along with ```rxqueues``` and ```txqueues```, the number of receiving and transmitting queues as assigned in the CLI script while listing the ethernet devices. The second line shows the port number ```port#``` as specified in the I/O file. The remaining lines of the output show the received packet count ```RX packets``` and the transmitted packet count ```TX packets``` with the corresponding total byte-count bytes and the number of dropped packets ```misses``` for each.

### Step 5.5.4: Closing telnet session

Close the pipeline CLI and the telnet session by pressing ```ctrl+]```, press ```Enter```, and then type the ```quit``` command.

# Step 6: Building the experiment topology
This section shows the steps required to build the experiment topology shown in the lab topology section. This will be done by creating two Linux namespaces and configuring the interfaces so that a connection between them is established.

Namespaces are a feature that partitions Linux resources. Linux namespaces provide independent instances of networks that enable network isolation and independent operations. Each network namespace has its own networking devices, IP addresses, routing tables, and firewall rules [<a href="#References">9</a>]. 

### Step 6.1: Create namespaces
The following commands create namespaces h1 and h2.

<img src="./labs_files/lab1/figs/6_01_namespaces.png" width="550px"><br>

In [17]:
stdout, stderr = server1.execute(f'sudo ip netns add h1')
stdout, stderr = server1.execute(f'sudo ip netns add h2')

 sudo: unable to resolve host server1: Name or service not known
  sudo: unable to resolve host server1: Name or service not known
 

### Step 6.2: Attaching virtual device interfaces to namespaces
The following commands will attach dtap0 to namespace h1 and dtap1 to namespace h2.

<img src="./labs_files/lab1/figs/6_02_dtap.png" width="550px"><br>

In [18]:
stdout, stderr = server1.execute(f'sudo ip link set dtap0 netns h1')
stdout, stderr = server1.execute(f'sudo ip link set dtap1 netns h2')

 sudo: unable to resolve host server1: Name or service not known
  sudo: unable to resolve host server1: Name or service not known
 

### Step 6.3: Activating virtual device interfaces
The following commands will turn up dtap0 on namespace h1 and dtap1 on namespace h2.

<img src="./labs_files/lab1/figs/6_03_up.png" width="550px"><br>

In [19]:
stdout, stderr = server1.execute(f'sudo ip netns exec h1 ip link set dev dtap0 up')
stdout, stderr = server1.execute(f'sudo ip netns exec h2 ip link set dev dtap1 up')

 sudo: unable to resolve host server1: Name or service not known
  sudo: unable to resolve host server1: Name or service not known
 

### Step 6.4: Assigning IP addresses to the interfaces
The following commands will assign the IP address 192.168.10.1 to dtap0 on namespace h1 and 192.168.10.2 to dtap1 on namespace h2.

<img src="./labs_files/lab1/figs/6_04_ip.png" width="550px"><br>

In [20]:
stdout, stderr = server1.execute(f'sudo ip netns exec h1 ifconfig dtap0 192.168.10.1/24')
stdout, stderr = server1.execute(f'sudo ip netns exec h2 ifconfig dtap1 192.168.10.2/24')

 sudo: unable to resolve host server1: Name or service not known
  sudo: unable to resolve host server1: Name or service not known
 

### Step 6.5: Establishing connection between namespaces
The following commands add a static ARP entry to the ARP cache in h1 and h2 to associate the IP addresses in the namespaces to the corresponding MAC addresses.

In [21]:
stdout, stderr = server1.execute(f'sudo ip netns exec h1 arp -s 192.168.10.2 00:00:00:00:00:02')
stdout, stderr = server1.execute(f'sudo ip netns exec h2 arp -s 192.168.10.1 00:00:00:00:00:01')

 sudo: unable to resolve host server1: Name or service not known
  sudo: unable to resolve host server1: Name or service not known
 

# Step 7: Testing connectivity

### Step 7.1: Sending packets from h1 to h2
Test the connectivity between namespaces h1 and h2 using the ping command.

In [22]:
stdout, stderr = server1.execute(f'sudo ip netns exec h1 ping 192.168.10.2 -c 4')

PING 192.168.10.2 (192.168.10.2) 56(84) bytes of data.
64 bytes from 192.168.10.2: icmp_seq=1 ttl=64 time=0.131 ms
 sudo: unable to resolve host server1: Name or service not known
 64 bytes from 192.168.10.2: icmp_seq=2 ttl=64 time=0.081 ms
64 bytes from 192.168.10.2: icmp_seq=3 ttl=64 time=0.078 ms
64 bytes from 192.168.10.2: icmp_seq=4 ttl=64 time=12.2 ms

--- 192.168.10.2 ping statistics ---
4 packets transmitted, 4 received, 0% packet loss, time 3075ms
rtt min/avg/max/mdev = 0.078/3.110/12.151/5.219 ms


### Step 7.2: Stopping the DPDK pipeline
Stop the DPDK pipeline by pressing ```ctrl+c``` in the terminal running the pipeline.

<img src="./labs_files/lab1/figs/7_02.png" width="550px"><br>

Note that the logs in the terminal correspond to the code executed for packet processing. 

# Step 8: Delete the slice

This concludes Lab 1. Please delete your slice when you are done with your experiment.

In [23]:
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager
fablib = fablib_manager()
slice = fablib.get_slice(name="P4DPDK_lab1_vnic")
slice.delete()

# References

1.	H. Zhu, “Data Plane Development Kit (DPDK): A Software Optimization Guide to the User Space-based Network Application”, CRC Press, 2020.
2.	R. Donata, “What is DPDK?”, [Online]. Available: https://tinyurl.com/yfc73h7c.
3.	DPDK, “Data Plane Development Kit documentation”, Release 2.2.0, 2016.
4.	Intel, “Introduction to the Data Plane Development Kit (DPDK) Packet Framework”, [Online]. Available: https://tinyurl.com/254r9sc5.
5.	DPDK, “rte_pipeline.h File Reference”, [Online]. Available: https://tinyurl.com/sh9254cs.
6.	S. Ibanez, “The p4-> netfpga workflow for line-rate packet processing”, Proceedings of the 2019 ACM/SIGDA International Symposium on Field-Programmable Gate Arrays, 2019.
7.	P4lang, “DPDK Backend”, [Online]. Available: https://tinyurl.com/cw29ubxa.
8.	DPDK, “Memory in DPDK, Part 1: General Concepts”, [Online]. Available: https://www.dpdk.org/memory-in-dpdk-part-1-general-concepts/.
9.	Toptal, “Separation anxiety: A tutorial for isolating your system with Linux namespaces”, [Online]. Available: https://tinyurl.com/3wm32dbd.